<a href="https://colab.research.google.com/github/ThatManRong/RL_Thesis/blob/main/RealData_SingleAsset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl

In [ ]:
# !pip install yfinance

In [31]:
import yfinance as yf
start_date = '2005-09-30'
end_date = '2020-09-30'
data = yf.download("SPY", start=start_date, end=end_date)
data["Bid"] = data["Close"]*0.999
data["Ask"] = data["Close"]*1.001
#data = data.set_index('Date')
# print(data.index.min(), data.index.max())
data.head()
# print(data[:]['Adj Close'])
# SPY_price = data[:]['Adj Close']
# SPY_price = SPY_price[0:len(SPY_price):250]
# SPY_return = SPY_price.pct_change().dropna()
# SPY_return = SPY_return.values.tolist()
# SPY_act_return = []
# base = 1
# for i in range(len(SPY_return)):
#   if i == 0:
#     SPY_act_return.append(base)
#   else:
#     SPY_act_return.append(SPY_act_return[i-1]*(1+SPY_return[i]))

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Bid,Ask
Date,,,,,,,,
2005-09-30,122.620003,123.040001,121.739998,123.040001,90.895706,47824200,122.916961,123.163041
2005-10-03,122.959999,123.339996,122.449997,122.599998,90.570633,50994800,122.477398,122.722598
2005-10-04,122.790001,123.029999,121.160004,121.220001,89.551186,60776300,121.098781,121.341221
2005-10-05,121.250000,121.309998,119.570000,119.629997,88.376595,106052100,119.510367,119.749627
2005-10-06,119.779999,120.260002,118.169998,119.199997,88.058876,140941800,119.080797,119.319197


In [32]:
date_split = '2016-01-01'
train = data[:date_split]
test = data[date_split:]
len(train),len(test)

(2581, 1194)

In [12]:
def plot_train_test(train, test, date_split):
    
    data = [
        Candlestick(x=train.index, open=train['Open'], high=train['High'], low=train['Low'], close=train['Close'], name='train'),
        Candlestick(x=test.index, open=test['Open'], high=test['High'], low=test['Low'], close=test['Close'], name='test')
    ]
    layout = {
         'shapes': [
             {'x0': date_split, 'x1': date_split, 'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper', 'line': {'color': 'rgb(0,0,0)', 'width': 1}}
         ],
        'annotations': [
            {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'left', 'text': ' test data'},
            {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'right', 'text': 'train data '}
        ]
    }
    figure = Figure(data=data, layout=layout)
    iplot(figure)

In [15]:
plot_train_test(train, test, date_split)

#State: 
>$S_t=(X_t, P_t) $

where X is inventory and P is price

#Action: 
>$a_t = \{Buy, Sell, Hold\} $

#State Evolution
>$X_{t+1}= X_t + a_t $

>$P_{t+1}= f(P_t) + \epsilon $

In [33]:
class Environment1:
    
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs
    
    def step(self, act):
        reward = 0
        
        # act = 0: stay, 1: buy, 2: sell
        if act == 1:
            self.positions.append(self.data.iloc[self.t, :]['Ask'])
        elif act == 2: # sell
            if len(self.positions) == 0:
                reward = -1
            else:
                profits = 0
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Bid'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['Bid'] - p)
        self.history.pop(0)
        average = self.data.iloc[(self.t-1), :]['Ask']
        if self.t >=10:
          average = np.mean(self.data.iloc[(self.t-10):(self.t-1), :]['Ask'])
        self.history.append(self.data.iloc[self.t, :]['Bid'] - average)
        if (self.t==len(self.data)-1):
            self.done=True
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done

In [ ]:
env = Environment1(train)
print(env.reset())
for _ in range(3):
    pact = np.random.randint(3)
    print (pact)
    print(env.step(pact))

In [35]:
class Q_Network(nn.Module):
        
    def __init__(self,obs_len,hidden_size,actions_n):
            
        super(Q_Network,self).__init__()
            
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
        
        
    def forward(self,x):
        h =  self.fc_val(x)
        return (h)

In [36]:
hidden_size=100
input_size=env.history_t+1
output_size=3
USE_CUDA = False
LR = 0.001

In [37]:
Q = Q_Network(input_size, hidden_size, output_size)
Q_ast = copy.deepcopy(Q)


In [62]:
if USE_CUDA:
    Q = Q.cuda()
loss_function = nn.MSELoss()
optimizer = optim.Adam(list(Q.parameters()), lr=LR)
epoch_num = 200
step_max = len(env.data)-1
memory_size = 200
batch_size = 200
gamma = 0.97
memory = []
total_step = 0
total_rewards = []
total_losses = []
epsilon = 1.0
epsilon_decrease = 1e-3
epsilon_min = 0.1
start_reduce_epsilon = 200
train_freq = 5
update_q_freq = 10
gamma = 0.97
show_log_freq = 20

In [ ]:
start = time.time()
for epoch in range(epoch_num):

    pobs = env.reset()
    step = 0
    done = False
    total_reward = 0
    total_loss = 0
    print("Percentage done: ", 100*epoch/epoch_num)
    while not done and step < step_max:

        # select act
        pact = np.random.randint(3)
        if np.random.rand() > epsilon:
            pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
            pact = np.argmax(pact.data)
            pact = pact.numpy()

        # act
        obs, reward, done = env.step(pact)

        # add memory
        memory.append((pobs, pact, reward, obs, done))
        if len(memory) > memory_size:
            memory.pop(0)

        # train or update q
        if len(memory) == memory_size:
            if total_step % train_freq == 0:
                shuffled_memory = np.random.permutation(memory)
                memory_idx = range(len(shuffled_memory))
                for i in memory_idx[::batch_size]:
                    batch = np.array(shuffled_memory[i:i+batch_size])
                    b_pobs = np.array(batch[:, 0].tolist(), dtype=np.float32).reshape(batch_size, -1)
                    b_pact = np.array(batch[:, 1].tolist(), dtype=np.int32)
                    b_reward = np.array(batch[:, 2].tolist(), dtype=np.int32)
                    b_obs = np.array(batch[:, 3].tolist(), dtype=np.float32).reshape(batch_size, -1)
                    b_done = np.array(batch[:, 4].tolist(), dtype=np.bool)

                    q = Q(torch.from_numpy(b_pobs))
                    q_ = Q_ast(torch.from_numpy(b_obs))
                    maxq = np.max(q_.data.numpy(),axis=1)
                    target = copy.deepcopy(q.data)
                    for j in range(batch_size):
                        target[j, b_pact[j]] = b_reward[j]+gamma*maxq[j]*(not b_done[j])
                    Q.zero_grad()
                    loss = loss_function(q, target)
                    total_loss += loss.data.item()
                    loss.backward()
                    optimizer.step()
                    
            if total_step % update_q_freq == 0:
                Q_ast = copy.deepcopy(Q)
                
            # epsilon
            if epsilon > epsilon_min and total_step > start_reduce_epsilon:
                epsilon -= epsilon_decrease

            # next step
            total_reward += reward
            pobs = obs
            step += 1
            total_step += 1

        total_rewards.append(total_reward)
        total_losses.append(total_loss)

        if (epoch+1) % show_log_freq == 0:
            log_reward = sum(total_rewards[((epoch+1)-show_log_freq):])/show_log_freq
            log_loss = sum(total_losses[((epoch+1)-show_log_freq):])/show_log_freq
            elapsed_time = time.time()-start
            print('\t'.join(map(str, [epoch+1, epsilon, total_step, log_reward, log_loss, elapsed_time])))
            start = time.time()
            
#return Q, total_losses, total_rewards

In [69]:

test_acts = []
test_rewards = []
test_profits = []
for i in range(1):
  test_env = Environment1(test)
  pobs = test_env.reset()
  #print("Percentage done: ", i*100/1000)

  for _ in range(len(test_env.data)-1):
      
      pact = Q(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
      pact = np.argmax(pact.data)
      test_acts.append(pact.item())
              
      obs, reward, done = test_env.step(pact.numpy())
      test_rewards.append(reward)

      pobs = obs
      #test_profits
  test_profits.append(test_env.profits)
  #test_profits = test_env.profits

In [70]:
print("Average Profits: ", np.mean(test_profits))

Average Profits:  374.21086929323764
